In [1]:
import pandas as pd
import numpy as np

In [2]:
print(pd.__version__)
print(np.__version__)

1.4.3
1.21.2


##### points to notice



- pd.eval()
- pd.query()
- df.eval()
- df.query()

As we’ve already seen in previous chapters, the power of the PyData stack is built
upon the ability of NumPy and Pandas to push basic operations into C via an intu‐
itive syntax: examples are vectorized/broadcasted operations in NumPy, and
grouping-type operations in Pandas. While these abstractions are efficient and effec‐ tive for many common use cases, they often rely on the creation of temporary inter‐
mediate objects, which can cause undue overhead in computational time and
memory use. <br>

As of version 0.13 (released January 2014), Pandas includes some experimental tools
that allow you to directly access C-speed operations without costly allocation of inter‐
mediate arrays. These are the eval() and query() functions, which rely on the
Numexpr package. In this notebook we will walk through their use and give some
rules of thumb about when you might think about using them.

##### Motivating query() and eval(): Compound Expressions

We’ve seen previously that NumPy and Pandas support fast vectorized operations; for
example, when you are adding the elements of two arrays:

In [7]:
rng = np.random.RandomState(42)
x = rng.randint(1E6)
y = rng.randint(1E6)
%timeit x + y

51.4 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


##### pandas.eval() for Efficient Operations

The eval() function in Pandas uses string expressions to efficiently compute opera‐
tions using DataFrames. For example, consider the following DataFrames:

In [9]:
nrows, ncols = 100000, 10
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                             for i in range(4))

To compute the sum of all four DataFrames using the typical Pandas approach, we can
just write the sum:

In [10]:
%timeit df1 + df2 + df3 + df4

13.9 ms ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can compute the same result via pd.eval by constructing the expression as a
string:

In [11]:
%timeit pd.eval('df1 + df2 + df3 + df4')

14.7 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
np.allclose(df1 + df2 + df3 + df4,
                   pd.eval('df1 + df2 + df3 + df4'))

True

##### Operations supported by pd.eval()

In [13]:
 df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                                   for i in range(5))

**Arithmetic operators.**    pd.eval() supports all arithmetic operators. For example:

In [17]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

**Comparison operators.**    pd.eval() supports all comparison operators, including
chained expressions:

In [18]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

**Bitwise operators.**    pd.eval() supports the & and | bitwise operators:

In [20]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In addition, it supports the use of the literal and and or in Boolean expressions:

In [21]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

**Object attributes and indices.**    pd.eval() supports access to object attributes via the
obj.attr syntax, and indexes via the obj[index] syntax:

In [22]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

**Other operations.**    Other operations, such as function calls, conditional statements,
loops, and other more involved constructs, are currently not implemented in
pd.eval(). If you’d like to execute these more complicated types of expressions, you
can use the Numexpr library itself.

##### DataFrame.eval() for Column-Wise Operations

Just as Pandas has a top-level pd.eval() function, DataFrames have an eval()
method that works in similar ways. The benefit of the eval() method is that columns
can be referred to by name. We’ll use this labeled array as an example:

In [23]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.259905,0.861337,0.232742
1,0.113747,0.154223,0.778527
2,0.460417,0.908127,0.264339
3,0.741817,0.243234,0.337768
4,0.449753,0.748700,0.057189


Using pd.eval() as above, we can compute expressions with the three columns like
this:

In [24]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

The DataFrame.eval() method allows much more succinct evaluation of expressions
with the columns:

In [25]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

Notice here that we treat column names as variables within the evaluated expression,
and the result is what we would wish.

##### Assignment in DataFrame.eval()

In addition to the options just discussed, DataFrame.eval() also allows assignment
to any column. Let’s use the DataFrame from before, which has columns 'A', 'B', and
'C':

In [26]:
df.head()

,A,B,C
0,0.259905,0.861337,0.232742
1,0.113747,0.154223,0.778527
2,0.460417,0.908127,0.264339
3,0.741817,0.243234,0.337768
4,0.449753,0.748700,0.057189


We can use df.eval() to create a new column 'D' and assign to it a value computed
from the other columns:

In [28]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.259905,0.861337,0.232742,4.817534
1,0.113747,0.154223,0.778527,0.344202
2,0.460417,0.908127,0.264339,5.177240
3,0.741817,0.243234,0.337768,2.916348
4,0.449753,0.748700,0.057189,20.955826


In the same way, any existing column can be modified:

In [29]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.259905,0.861337,0.232742,-2.584112
1,0.113747,0.154223,0.778527,-0.051990
2,0.460417,0.908127,0.264339,-1.693699
3,0.741817,0.243234,0.337768,1.476108
4,0.449753,0.748700,0.057189,-5.227312


##### Local variables in DataFrame.eval()

The DataFrame.eval() method supports an additional syntax that lets it work with
local Python variables. Consider the following:

In [30]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

The @ character here marks a variable name rather than a column name, and lets you
efficiently evaluate expressions involving the two “namespaces”: the namespace of
columns, and the namespace of Python objects. Notice that this @ character is only
supported by the DataFrame.eval() method, not by the pandas.eval() function,
because the pandas.eval() function only has access to the one (Python) namespace.

##### DataFrame.query() Method

The DataFrame has another method based on evaluated strings, called the query()
method. Consider the following:

In [31]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

As with the example used in our discussion of DataFrame.eval(), this is an expres‐
sion involving columns of the DataFrame. It cannot be expressed using the Data
Frame.eval() syntax, however! Instead, for this type of filtering operation, you can
use the query() method:

In [32]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In addition to being a more efficient computation, compared to the masking expres‐
sion this is much easier to read and understand. Note that the query() method also
accepts the @ flag to mark local variables:

In [33]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

##### Performance: When to Use These Functions

When considering whether to use these functions, there are two considerations: com‐
putation time and memory use. Memory use is the most predictable aspect. As already
mentioned, every compound expression involving NumPy arrays or Pandas Data
Frames will result in implicit creation of temporary arrays: For example, this:

In [40]:
x1 = df[(df.A < 0.5) & (df.B < 0.5)]
x1.head()

,A,B,C,D
1,0.113747,0.154223,0.778527,-0.051990
6,0.115987,0.119138,0.486502,-0.006476
10,0.083683,0.192117,0.170891,-0.634523
11,0.458644,0.006565,0.850275,0.531686
12,0.388111,0.492187,0.421714,-0.246794


is roughly equivalent to this:

In [39]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]
x.head()

,A,B,C,D
1,0.113747,0.154223,0.778527,-0.051990
6,0.115987,0.119138,0.486502,-0.006476
10,0.083683,0.192117,0.170891,-0.634523
11,0.458644,0.006565,0.850275,0.531686
12,0.388111,0.492187,0.421714,-0.246794


If the size of the temporary DataFrames is significant compared to your available sys‐
tem memory (typically several gigabytes), then it’s a good idea to use an eval() or
query() expression. You can check the approximate size of your array in bytes using
this:

In [42]:
print("X1:", x1.values.nbytes)
print("X:", x.values.nbytes)
print(print("df:", df.values.nbytes))

X1: 8000
X: 8000
df: 32000
None


On the performance side, eval() can be faster even when you are not maxing out
your system memory. The issue is how your temporary DataFrames compare to the
size of the L1 or L2 CPU cache on your system (typically a few megabytes in 2016); if
they are much bigger, then eval() can avoid some potentially slow movement of val‐
ues between the different memory caches. In practice, I find that the difference in
computation time between the traditional methods and the eval/query method is
usually not significant—if anything, the traditional method is faster for smaller
arrays! The benefit of eval/query is mainly in the saved memory, and the sometimes
cleaner syntax they offer. <br>

We’ve covered most of the details of eval() and query() here; for more information
on these, you can refer to the Pandas documentation. In particular, different parsers
and engines can be specified for running these queries; for details on this, see the dis‐
cussion within the “Enhancing Performance” section.